**Identificar como o tempo está distribuido no espaço**


#Instalando bibliotecas

In [11]:
 !pip install 'fsspec>=0.3.3'

In [12]:
!pip install unidecode

#Importando Bibliotecas e arquivos do drive

In [13]:
from unidecode import unidecode
import dask.dataframe as dd
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!ls "/content/drive/My Drive"

 ATESTADO_MATRICULA__GRADUACAO2014057.pdf
 Certificados
'Colab Notebooks'
 Contas.gsheet
 Curriculo
 FEMU
 ImersaoDev_Alura
'Iniciação Científica'
'Meet Recordings'
 Obra
 UFOP
'Voluta '
'WhatsApp Image 2022-03-17 at 10.17.42.jpeg'
 Wizard


##Lendo os dados

In [20]:
df = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/olist_new_df.csv')

In [21]:
df.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id', 'price',
       'freight_value', 'payment_type', 'payment_installments',
       'payment_value', 'customer_unique_id', 'product_category_name',
       'product_weight_g', 'product_length_cm', 'product_height_cm',
       'product_width_cm', 'costumer_lat', 'costumer_lng', 'seller_lat',
       'seller_lng', 'days_to_delivery', 'early_delivery', 'day_shipping',
       'month_shipping', 'year_shipping', 'day_order_approved',
       'month_order_approved', 'year_order_approved'],
      dtype='object')

In [22]:
df = df.compute()

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111237 entries, 0 to 111236
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   order_id               111237 non-null  float64
 1   order_item_id          111237 non-null  float64
 2   product_id             111237 non-null  float64
 3   seller_id              111237 non-null  float64
 4   price                  111237 non-null  float64
 5   freight_value          111237 non-null  float64
 6   payment_type           111237 non-null  float64
 7   payment_installments   111237 non-null  float64
 8   payment_value          111237 non-null  float64
 9   customer_unique_id     111237 non-null  float64
 10  product_category_name  111237 non-null  float64
 11  product_weight_g       111237 non-null  float64
 12  product_length_cm      111237 non-null  float64
 13  product_height_cm      111237 non-null  float64
 14  product_width_cm       111237 non-nu

#Visualizações do tempo no espaço





##Mapas de calor 


In [ ]:
fig = px.density_mapbox(df, lat='costumer_lat', lon='costumer_lng', z='days_to_delivery', radius=10,
                        center=dict(lat=0, lon=180), zoom=0, height=1000,
                        mapbox_style="stamen-terrain")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig = px.density_mapbox(df, lat='seller_lat', lon='seller_lng', z='days_to_delivery', radius=10,
                        center=dict(lat=0, lon=180), zoom=0, height=1000,
                        mapbox_style="stamen-terrain")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

##Adicionando costumer city e costumer state ao dataframe


In [24]:
df_aux = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/olist_geolocated_transactions.csv')

In [25]:
df_aux = df_aux.compute()

In [26]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117729 entries, 0 to 14207
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       117729 non-null  object 
 1   order_item_id                  117729 non-null  float64
 2   product_id                     117729 non-null  object 
 3   seller_id                      117729 non-null  object 
 4   shipping_limit_date            117729 non-null  object 
 5   price                          117729 non-null  float64
 6   freight_value                  117729 non-null  float64
 7   payment_sequential             117726 non-null  float64
 8   payment_type                   117726 non-null  object 
 9   payment_installments           117726 non-null  float64
 10  payment_value                  117726 non-null  float64
 11  review_id                      116753 non-null  object 
 12  review_score                   

In [27]:
df_aux.head(10)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,costumer_lat,costumer_lng,seller_lat,seller_lng
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,1.0,credit_card,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-21.763186,-41.310265,-22.497188,-44.127324
1,130898c0987d1801452a8ed92a670612,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,1.0,boleto,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-17.884803,-51.717167,-22.497188,-44.127324
2,532ed5e14e24ae1f0d735b91524b98b9,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,1.0,credit_card,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-19.913964,-43.976106,-22.497188,-44.127324
3,6f8c31653edb8c83e1a739408b5ff750,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.9,16.17,1.0,credit_card,3.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-25.520334,-49.124988,-22.497188,-44.127324
4,7d19f4ef4d04461989632411b7e588b9,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.9,13.29,1.0,credit_card,4.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-20.655379,-43.776331,-22.497188,-44.127324
5,a0f9acf0b6294ed8561e32cde1a966bc,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-03 15:23:46,55.9,26.93,1.0,credit_card,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-12.906138,-39.192076,-22.497188,-44.127324
6,bbf796534aaf9c59f8da8c7982db56e0,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-03-26 03:28:44,64.9,38.50,1.0,boleto,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-15.036469,-41.260324,-22.497188,-44.127324
7,c9ef97d2854afe64a3b4488bc2836af6,2.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-14 04:25:34,58.9,18.12,1.0,boleto,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-17.782743,-50.917956,-22.497188,-44.127324
8,c9ef97d2854afe64a3b4488bc2836af6,1.0,436c8d57ff8d4aa254318e9bd9b48c83,48436dade18ac8b2bce089ec2a041202,2017-08-14 04:25:34,58.9,17.83,1.0,boleto,1.0,...,599.0,6.0,530.0,30.0,9.0,14.0,-17.782743,-50.917956,-22.497188,-44.127324
9,df2d910b8b5e5f461f67043489f9569d,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-06-09 14:31:21,55.9,35.71,2.0,voucher,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-7.957598,-34.874754,-22.497188,-44.127324


In [28]:
df_aux.drop_duplicates(keep=False,inplace=True)

In [29]:
df_aux.head(10)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,costumer_lat,costumer_lng,seller_lat,seller_lng
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,1.0,credit_card,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-21.763186,-41.310265,-22.497188,-44.127324
1,130898c0987d1801452a8ed92a670612,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,1.0,boleto,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-17.884803,-51.717167,-22.497188,-44.127324
2,532ed5e14e24ae1f0d735b91524b98b9,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,1.0,credit_card,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-19.913964,-43.976106,-22.497188,-44.127324
3,6f8c31653edb8c83e1a739408b5ff750,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.9,16.17,1.0,credit_card,3.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-25.520334,-49.124988,-22.497188,-44.127324
4,7d19f4ef4d04461989632411b7e588b9,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.9,13.29,1.0,credit_card,4.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-20.655379,-43.776331,-22.497188,-44.127324
5,a0f9acf0b6294ed8561e32cde1a966bc,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-03 15:23:46,55.9,26.93,1.0,credit_card,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-12.906138,-39.192076,-22.497188,-44.127324
6,bbf796534aaf9c59f8da8c7982db56e0,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-03-26 03:28:44,64.9,38.50,1.0,boleto,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-15.036469,-41.260324,-22.497188,-44.127324
7,c9ef97d2854afe64a3b4488bc2836af6,2.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-14 04:25:34,58.9,18.12,1.0,boleto,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-17.782743,-50.917956,-22.497188,-44.127324
8,c9ef97d2854afe64a3b4488bc2836af6,1.0,436c8d57ff8d4aa254318e9bd9b48c83,48436dade18ac8b2bce089ec2a041202,2017-08-14 04:25:34,58.9,17.83,1.0,boleto,1.0,...,599.0,6.0,530.0,30.0,9.0,14.0,-17.782743,-50.917956,-22.497188,-44.127324
9,df2d910b8b5e5f461f67043489f9569d,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-06-09 14:31:21,55.9,35.71,2.0,voucher,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-7.957598,-34.874754,-22.497188,-44.127324


In [30]:
df_aux = df_aux.drop(columns=['review_comment_message','review_comment_title','review_score',
                              'review_id','payment_value','payment_installments','payment_type','payment_sequential',
                              'freight_value','price','shipping_limit_date','seller_id','product_id','order_item_id'])

In [31]:
df_aux = df_aux.drop(columns=['seller_zip_code_prefix','customer_zip_code_prefix','customer_unique_id',
                              'order_estimated_delivery_date','order_delivered_customer_date',
                              'order_delivered_carrier_date','order_approved_at','order_purchase_timestamp',
                              'order_status','customer_id','review_answer_timestamp','review_creation_date'])

In [32]:
df_aux = df_aux.drop(columns=['seller_lng','seller_lat','costumer_lng','costumer_lat','product_width_cm',
                              'product_height_cm','product_length_cm','product_weight_g','product_photos_qty',
                              'product_description_lenght','product_name_lenght','product_category_name'])

In [33]:
df_aux.head()

,order_id,customer_city,customer_state,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,campos dos goytacazes,RJ,volta redonda,SP
1,130898c0987d1801452a8ed92a670612,jatai,GO,volta redonda,SP
2,532ed5e14e24ae1f0d735b91524b98b9,belo horizonte,MG,volta redonda,SP
3,6f8c31653edb8c83e1a739408b5ff750,sao jose dos pinhais,PR,volta redonda,SP
4,7d19f4ef4d04461989632411b7e588b9,conselheiro lafaiete,MG,volta redonda,SP


In [34]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df_aux["order_id"] = encoder.fit_transform(df_aux["order_id"])
df_aux.head()

,order_id,customer_city,customer_state,seller_city,seller_state
0,0,campos dos goytacazes,RJ,volta redonda,SP
1,7310,jatai,GO,volta redonda,SP
2,31905,belo horizonte,MG,volta redonda,SP
3,42787,sao jose dos pinhais,PR,volta redonda,SP
4,47910,conselheiro lafaiete,MG,volta redonda,SP


In [35]:
df_aux.head(10)

,order_id,customer_city,customer_state,seller_city,seller_state
0,0,campos dos goytacazes,RJ,volta redonda,SP
1,7310,jatai,GO,volta redonda,SP
2,31905,belo horizonte,MG,volta redonda,SP
3,42787,sao jose dos pinhais,PR,volta redonda,SP
4,47910,conselheiro lafaiete,MG,volta redonda,SP
5,61385,dom macedo costa,BA,volta redonda,SP
6,72034,belo campo,BA,volta redonda,SP
7,77413,rio verde,GO,volta redonda,SP
8,77413,rio verde,GO,volta redonda,SP
9,85737,paulista,PE,volta redonda,SP


In [36]:
df_aux.isnull().sum()

order_id          0
customer_city     0
customer_state    0
seller_city       0
seller_state      0
dtype: int64

In [37]:
db = pd.merge(df_aux, df, on='order_id')

In [38]:
db.head()

,order_id,customer_city,customer_state,seller_city,seller_state,order_item_id,product_id,seller_id,price,freight_value,...,seller_lat,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved
0,0,campos dos goytacazes,RJ,volta redonda,SP,1.0,8216.0,812.0,58.90,13.29,...,-22.497188,-44.127324,6.0,10.0,19.0,9.0,2017.0,13.0,9.0,2017.0
1,7310,jatai,GO,volta redonda,SP,1.0,28330.0,617.0,72.00,17.78,...,-16.690438,-49.327015,0.0,22.0,10.0,10.0,2017.0,4.0,10.0,2017.0
2,31905,belo horizonte,MG,volta redonda,SP,1.0,28451.0,1444.0,99.99,17.43,...,-23.486138,-46.367107,5.0,12.0,30.0,8.0,2017.0,18.0,8.0,2017.0
3,42787,sao jose dos pinhais,PR,volta redonda,SP,1.0,29612.0,1901.0,69.90,9.34,...,-23.490646,-46.476651,7.0,9.0,14.0,2.0,2018.0,10.0,2.0,2018.0
4,42787,sao jose dos pinhais,PR,volta redonda,SP,1.0,29612.0,1901.0,69.90,9.34,...,-23.490646,-46.476651,7.0,9.0,14.0,2.0,2018.0,10.0,2.0,2018.0


In [39]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133371 entries, 0 to 133370
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   order_id               133371 non-null  int64  
 1   customer_city          133371 non-null  object 
 2   customer_state         133371 non-null  object 
 3   seller_city            133371 non-null  object 
 4   seller_state           133371 non-null  object 
 5   order_item_id          133371 non-null  float64
 6   product_id             133371 non-null  float64
 7   seller_id              133371 non-null  float64
 8   price                  133371 non-null  float64
 9   freight_value          133371 non-null  float64
 10  payment_type           133371 non-null  float64
 11  payment_installments   133371 non-null  float64
 12  payment_value          133371 non-null  float64
 13  customer_unique_id     133371 non-null  float64
 14  product_category_name  133371 non-nu

In [40]:
db.drop_duplicates(keep=False,inplace=True)

In [41]:
db.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 96569 entries, 0 to 133370
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   order_id               96569 non-null  int64  
 1   customer_city          96569 non-null  object 
 2   customer_state         96569 non-null  object 
 3   seller_city            96569 non-null  object 
 4   seller_state           96569 non-null  object 
 5   order_item_id          96569 non-null  float64
 6   product_id             96569 non-null  float64
 7   seller_id              96569 non-null  float64
 8   price                  96569 non-null  float64
 9   freight_value          96569 non-null  float64
 10  payment_type           96569 non-null  float64
 11  payment_installments   96569 non-null  float64
 12  payment_value          96569 non-null  float64
 13  customer_unique_id     96569 non-null  float64
 14  product_category_name  96569 non-null  float64
 15  p

In [42]:
db.isnull().sum()

order_id                 0
customer_city            0
customer_state           0
seller_city              0
seller_state             0
order_item_id            0
product_id               0
seller_id                0
price                    0
freight_value            0
payment_type             0
payment_installments     0
payment_value            0
customer_unique_id       0
product_category_name    0
product_weight_g         0
product_length_cm        0
product_height_cm        0
product_width_cm         0
costumer_lat             0
costumer_lng             0
seller_lat               0
seller_lng               0
days_to_delivery         0
early_delivery           0
day_shipping             0
month_shipping           0
year_shipping            0
day_order_approved       0
month_order_approved     0
year_order_approved      0
dtype: int64

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111237 entries, 0 to 111236
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   order_id               111237 non-null  float64
 1   order_item_id          111237 non-null  float64
 2   product_id             111237 non-null  float64
 3   seller_id              111237 non-null  float64
 4   price                  111237 non-null  float64
 5   freight_value          111237 non-null  float64
 6   payment_type           111237 non-null  float64
 7   payment_installments   111237 non-null  float64
 8   payment_value          111237 non-null  float64
 9   customer_unique_id     111237 non-null  float64
 10  product_category_name  111237 non-null  float64
 11  product_weight_g       111237 non-null  float64
 12  product_length_cm      111237 non-null  float64
 13  product_height_cm      111237 non-null  float64
 14  product_width_cm       111237 non-nu

In [44]:
df.head(10)

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,seller_lat,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved
0,0.0,1.0,8216.0,812.0,58.9,13.29,1.0,2.0,72.19,48076.0,...,-22.497188,-44.127324,6.0,10.0,19.0,9.0,2017.0,13.0,9.0,2017.0
1,7014.0,1.0,8216.0,812.0,55.9,17.96,0.0,1.0,73.86,5649.0,...,-22.497188,-44.127324,6.0,21.0,5.0,7.0,2017.0,29.0,6.0,2017.0
2,30559.0,1.0,8216.0,812.0,64.9,18.33,1.0,2.0,83.23,18589.0,...,-22.497188,-44.127324,5.0,15.0,23.0,5.0,2018.0,18.0,5.0,2018.0
3,40954.0,1.0,8216.0,812.0,58.9,16.17,1.0,3.0,75.07,82561.0,...,-22.497188,-44.127324,1.0,23.0,7.0,8.0,2017.0,1.0,8.0,2017.0
4,45875.0,1.0,8216.0,812.0,58.9,13.29,1.0,4.0,72.19,4247.0,...,-22.497188,-44.127324,1.0,21.0,16.0,8.0,2017.0,10.0,8.0,2017.0
5,58794.0,1.0,8216.0,812.0,55.9,26.93,1.0,1.0,82.83,18290.0,...,-22.497188,-44.127324,5.0,22.0,3.0,8.0,2017.0,28.0,7.0,2017.0
6,69027.0,1.0,8216.0,812.0,64.9,38.50,0.0,1.0,103.40,17734.0,...,-22.497188,-44.127324,0.0,23.0,26.0,3.0,2018.0,20.0,3.0,2018.0
7,74190.0,2.0,8216.0,812.0,58.9,18.12,0.0,1.0,153.75,47060.0,...,-22.497188,-44.127324,2.0,27.0,14.0,8.0,2017.0,8.0,8.0,2017.0
8,74190.0,1.0,8359.0,812.0,58.9,17.83,0.0,1.0,153.75,47060.0,...,-22.497188,-44.127324,2.0,27.0,14.0,8.0,2017.0,8.0,8.0,2017.0
9,82148.0,1.0,8216.0,812.0,55.9,35.71,3.0,1.0,20.00,5361.0,...,-22.497188,-44.127324,2.0,29.0,9.0,6.0,2017.0,5.0,6.0,2017.0


##Malhas

In [45]:
#malha estados
import json

with open('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/estados.json') as response:
    counties = json.load(response)


In [46]:
#malha cidades
import json

with open('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/cidades.json') as response:
    cidades = json.load(response)


###Pré processamento 

In [47]:
#replace uppercase
for feature in cidades["features"]:
    feature["properties"]["name"] = (feature["properties"]["name"].lower())

In [48]:
#remove accents
for feature in cidades["features"]:
    feature["properties"]["name"] = unidecode(feature["properties"]["name"])

In [49]:
#verificando se funcionou
cidades["features"]

Output hidden; open in https://colab.research.google.com to view.

#Visualização para Costumer 
 

##City

###Sem ser pela media

In [ ]:
#Costumer 
import plotly.express as px

#mapa com todos os dados (nao é feito pela media)
fig = px.choropleth_mapbox(db, geojson=cidades, color='days_to_delivery', range_color=(0, 16),
                           locations='customer_city', featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

####Testando outro modelo de mapa 

In [ ]:
db.info()

In [ ]:
import plotly.graph_objects as go

db['text'] = db['customer_state'] + '' + db['customer_city'] + 'Days to delivery: ' + db['days_to_delivery'].astype(str)

fig = go.Figure(data=go.Scattergeo(
        lon = db['costumer_lng'],
        lat = db['costumer_lat'],
        text = db['text'],
        mode = 'markers',
        marker_color = db['days_to_delivery'],
        ))

fig.update_layout(
        title = 'Days to delivery',
        geo_scope='south america',
    )
fig.show()

####Fazendo filtro no mapa



######Mais de mil pedidos por cidade

In [ ]:
#verificando o formato do dataframe até aqui
db.head(10)

In [ ]:
#contando a quantidade de pedidos por cidade
#aqui contabiliza-se a quantidade de vezes que uma cidade aparece no dataframe.
orders_city = db.groupby(['customer_city']).count()

In [ ]:
#transformando em um pandas dataframe
orders_city = pd.DataFrame(orders_city)

In [ ]:
#verificando o formato do novo df
orders_city.info()

In [ ]:
orders_city.head()

Ótimo, funcionou.

In [ ]:
#verificando o maior valor
#nesse caso qual o maior numero de vezes apareceu uma cidade
max = orders_city.max()
max

In [ ]:
#removendo colunas desnecessarias, todas possuem os mesmos valores, 
#logo, apenas uma delas é suficiente.
orders_city = orders_city.drop(columns=['order_item_id','product_id','seller_id',
                            'price','freight_value','payment_type',
                            'payment_installments','payment_value',
                            'customer_unique_id','product_category_name',
                            'product_weight_g','product_length_cm','product_height_cm',
                            'product_width_cm','costumer_lat','costumer_lng','seller_lat',
                            'seller_lng','days_to_delivery','early_delivery','day_shipping',
                            'month_shipping','year_shipping','day_order_approved','month_order_approved',
                            'year_order_approved','customer_state','seller_city','seller_state'])

In [ ]:
#funcionou
orders_city.head()

In [ ]:
#renomeando a coluna para ficar mais apresentavel 
orders_city = orders_city.rename(columns={'order_id': 'value'})

In [ ]:
#adicionando a coluna com o numero de ocorrencias das cidades ao dataframe final
df_aux_orders = pd.merge(db, orders_city, how='inner', on='customer_city')

In [ ]:
df_aux_orders.head(10)

In [ ]:
#verificando as colunas do dataframe final
df_aux_orders.info()

In [ ]:
#verificando se existe algum dado nulo
df_aux_orders.isnull().sum()

In [ ]:
#criando filtro para remover colunas que nao atendem ao esperado
#nesse, caso deseja-se apenas as cidades com mais de mil pedidos
filtro  = df_aux_orders['value'] > 1000

#cria-se df com as cidades que possuem mais de mil pedidos 
df_milorders = df_aux_orders[filtro]

In [ ]:
df_milorders.info()

In [ ]:
df_milorders.head(10)

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(df_milorders, geojson=cidades, color='days_to_delivery', range_color=(0, 20),
                           locations=('customer_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3, hover_name="value")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

######Mais de quinhentos pedidos por cidade

In [ ]:
filtro  = df_aux_orders['value'] > 500
df_quinorders = df_aux_orders[filtro]

In [ ]:
df_quinorders.info()

In [ ]:
df_quinorders.head(10)

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(df_quinorders, geojson=cidades, color='days_to_delivery', range_color=(0, 20),
                           locations=('customer_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3,hover_name="value")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

######Mais de cem pedidos por cidade

In [ ]:
filtro  = df_aux_orders['value'] > 100
df_cemorders = df_aux_orders[filtro]

In [ ]:
df_cemorders.info()

In [ ]:
df_cemorders.head(10)

In [ ]:
x =df_cemorders[df_cemorders["customer_city"] == "atibaia"]

In [ ]:
x.describe()

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(df_cemorders, geojson=cidades, color='days_to_delivery', range_color=(0, 20),
                           locations=('customer_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3, hover_name="value")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

######Mais de vinte pedidos por cidade

In [ ]:
filtro  = df_aux_orders['value'] > 20
df_20orders = df_aux_orders[filtro]

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(df_20orders, geojson=cidades, color='days_to_delivery', range_color=(0, 20),
                           locations=('customer_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3, hover_name="value")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

###### Mapa para numero de pedidos por cidade

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(df_aux_orders, geojson=cidades, color='value', range_color=(0, 20),
                           locations=('customer_city'), featureidkey="properties.name", 
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3, hover_name="value")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

###Days to delivery pela media dos dias para uma cidade

In [ ]:
mean_days = df_aux_orders.groupby(['customer_city']).mean()

In [ ]:
#transformando em um pandas dataframe
mean_days = pd.DataFrame(mean_days)

In [ ]:
mean_days.head(5)

In [ ]:
mean_days['costumer_city'] = mean_days.index

In [ ]:
mean_days.head()

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(mean_days, geojson=cidades, color='days_to_delivery', range_color=(0, 20),
                           locations=('costumer_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#criando filtro para remover colunas que nao atendem ao esperado
#nesse, caso deseja-se apenas as cidades com mais de mil pedidos
filtro  = mean_days['value'] > 20

#cria-se df com as cidades que possuem mais de mil pedidos 
df_20orders_mean = mean_days[filtro]

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(df_20orders_mean, geojson=cidades, color='days_to_delivery', range_color=(0, 7),
                           locations=('costumer_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
mean_days.info()

##### Mapa para media de numero de pedidos por cidade

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(mean_days, geojson=cidades, color='value', range_color=(0, 20),
                           locations=('costumer_city'), featureidkey="properties.name", 
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

###Days to delivery pelo desvio padrão dos dias para uma cidade

In [ ]:
desvio = df_aux_orders.groupby(['customer_city']).std()

In [ ]:
desvio = pd.DataFrame(desvio)

In [ ]:
desvio.head()

In [ ]:
#verificando se existe algum dado nulo
desvio.isnull().sum()

In [ ]:
desvio.dropna(inplace=True)

In [ ]:
desvio.isnull().sum()

In [ ]:
desvio.head()

In [ ]:
desvio['costumer_city'] = desvio.index

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(desvio, geojson=cidades, color='days_to_delivery', range_color=(0, 10),
                           locations=('costumer_city'), featureidkey="properties.name", 
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

##State

In [ ]:
db.head()

####Fazendo filtro no mapa



###Days to delivery pela media dos dias para uma cidade

In [ ]:
mean_days_state = df_aux_orders.groupby(['customer_state']).mean()

In [ ]:
#transformando em um pandas dataframe
mean_days_state = pd.DataFrame(mean_days_state)

In [ ]:
mean_days_state.head(5)

In [ ]:
mean_days_state['costumer_state'] = mean_days_state.index

In [ ]:
mean_days_state.head()

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(mean_days_state, geojson=counties, color='days_to_delivery', range_color=(0, 20),
                           locations=('costumer_state'), featureidkey="properties.SIGLA", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

###Days to delivery pelo desvio padrão dos dias para estado

In [ ]:
desvio = df_aux_orders.groupby(['customer_state']).std()

In [ ]:
desvio = pd.DataFrame(desvio)

In [ ]:
desvio.head()

In [ ]:
#verificando se existe algum dado nulo
desvio.isnull().sum()

In [ ]:
desvio.dropna(inplace=True)

In [ ]:
desvio.isnull().sum()

In [ ]:
desvio.head()

In [ ]:
desvio['costumer_state'] = desvio.index

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(desvio, geojson=counties, color='days_to_delivery', range_color=(0, 10),
                           locations=('costumer_state'), featureidkey="properties.SIGLA", 
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#Visualização para Seller

 

##City

###Sem ser pela media

In [50]:
#Seller 
import plotly.express as px

#mapa com todos os dados (nao é feito pela media)
fig = px.choropleth_mapbox(db, geojson=cidades, color='days_to_delivery', range_color=(0, 16),
                           locations='seller_city', featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

####Fazendo filtro no mapa



######Mais de mil pedidos por cidade

In [ ]:
#verificando o formato do dataframe até aqui
db.head(10)

In [52]:
#contando a quantidade de pedidos por cidade
#aqui contabiliza-se a quantidade de vezes que uma cidade aparece no dataframe.
orders_city_seller = db.groupby(['seller_city']).count()

In [53]:
#transformando em um pandas dataframe
orders_city_seller = pd.DataFrame(orders_city_seller)

In [54]:
#verificando o formato do novo df
orders_city_seller.info()

<class 'pandas.core.frame.DataFrame'>
Index: 586 entries, 04482255 to xaxim
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   order_id               586 non-null    int64
 1   customer_city          586 non-null    int64
 2   customer_state         586 non-null    int64
 3   seller_state           586 non-null    int64
 4   order_item_id          586 non-null    int64
 5   product_id             586 non-null    int64
 6   seller_id              586 non-null    int64
 7   price                  586 non-null    int64
 8   freight_value          586 non-null    int64
 9   payment_type           586 non-null    int64
 10  payment_installments   586 non-null    int64
 11  payment_value          586 non-null    int64
 12  customer_unique_id     586 non-null    int64
 13  product_category_name  586 non-null    int64
 14  product_weight_g       586 non-null    int64
 15  product_length_cm      586 non-null 

In [55]:
orders_city_seller.head()

,order_id,customer_city,customer_state,seller_state,order_item_id,product_id,seller_id,price,freight_value,payment_type,...,seller_lat,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved
seller_city,,,,,,,,,,,,,,,,,,,,,
04482255,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
abadia de goias,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
afonso claudio,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
aguas claras df,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
alambari,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


Ótimo, funcionou.

In [56]:
#verificando o maior valor
#nesse caso qual o maior numero de vezes apareceu uma cidade
max = orders_city_seller.max()
max

order_id                 23641
customer_city            23641
customer_state           23641
seller_state             23641
order_item_id            23641
product_id               23641
seller_id                23641
price                    23641
freight_value            23641
payment_type             23641
payment_installments     23641
payment_value            23641
customer_unique_id       23641
product_category_name    23641
product_weight_g         23641
product_length_cm        23641
product_height_cm        23641
product_width_cm         23641
costumer_lat             23641
costumer_lng             23641
seller_lat               23641
seller_lng               23641
days_to_delivery         23641
early_delivery           23641
day_shipping             23641
month_shipping           23641
year_shipping            23641
day_order_approved       23641
month_order_approved     23641
year_order_approved      23641
dtype: int64

In [57]:
#removendo colunas desnecessarias, todas possuem os mesmos valores, 
#logo, apenas uma delas é suficiente.
orders_city_seller = orders_city_seller.drop(columns=['order_item_id','product_id','seller_id',
                            'price','freight_value','payment_type',
                            'payment_installments','payment_value',
                            'customer_unique_id','product_category_name',
                            'product_weight_g','product_length_cm','product_height_cm',
                            'product_width_cm','costumer_lat','costumer_lng','seller_lat',
                            'seller_lng','days_to_delivery','early_delivery','day_shipping',
                            'month_shipping','year_shipping','day_order_approved','month_order_approved',
                            'year_order_approved','seller_state','customer_city','customer_state'])

In [58]:
orders_city_seller.head()

,order_id
seller_city,
04482255,2
abadia de goias,1
afonso claudio,6
aguas claras df,1
alambari,8


In [59]:
#renomeando a coluna para ficar mais apresentavel 
orders_city_seller = orders_city_seller.rename(columns={'order_id': 'value'})

In [62]:
#adicionando a coluna com o numero de ocorrencias das cidades ao dataframe final
df_aux_orders_seller = pd.merge(db, orders_city_seller, how='inner', on='seller_city')

In [63]:
df_aux_orders_seller.head(10)

,order_id,customer_city,customer_state,seller_city,seller_state,order_item_id,product_id,seller_id,price,freight_value,...,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value
0,0,campos dos goytacazes,RJ,volta redonda,SP,1.0,8216.0,812.0,58.90,13.29,...,-44.127324,6.0,10.0,19.0,9.0,2017.0,13.0,9.0,2017.0,142
1,7310,jatai,GO,volta redonda,SP,1.0,28330.0,617.0,72.00,17.78,...,-49.327015,0.0,22.0,10.0,10.0,2017.0,4.0,10.0,2017.0,142
2,31905,belo horizonte,MG,volta redonda,SP,1.0,28451.0,1444.0,99.99,17.43,...,-46.367107,5.0,12.0,30.0,8.0,2017.0,18.0,8.0,2017.0,142
3,42787,sao jose dos pinhais,PR,volta redonda,SP,1.0,29612.0,1901.0,69.90,9.34,...,-46.476651,7.0,9.0,14.0,2.0,2018.0,10.0,2.0,2018.0,142
4,42787,sao jose dos pinhais,PR,volta redonda,SP,1.0,29612.0,1901.0,69.90,9.34,...,-46.476651,7.0,9.0,14.0,2.0,2018.0,10.0,2.0,2018.0,142
5,47910,conselheiro lafaiete,MG,volta redonda,SP,1.0,6737.0,350.0,49.90,41.11,...,-49.388730,0.0,30.0,13.0,6.0,2018.0,11.0,6.0,2018.0,142
6,61385,dom macedo costa,BA,volta redonda,SP,1.0,30039.0,1539.0,229.04,22.61,...,-46.515649,2.0,13.0,9.0,3.0,2018.0,5.0,3.0,2018.0,142
7,72034,belo campo,BA,volta redonda,SP,1.0,24720.0,839.0,294.00,75.79,...,-48.829541,3.0,5.0,4.0,6.0,2018.0,26.0,5.0,2018.0,142
8,572,duque de caxias,RJ,volta redonda,SP,1.0,30290.0,1089.0,109.90,24.05,...,-47.417097,2.0,20.0,6.0,4.0,2017.0,2.0,4.0,2017.0,142
9,580,sao leopoldo,RS,volta redonda,SP,1.0,17184.0,1623.0,738.90,25.62,...,-46.573545,1.0,32.0,11.0,4.0,2017.0,5.0,4.0,2017.0,142


In [64]:
#verificando as colunas do dataframe final
df_aux_orders_seller.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96569 entries, 0 to 96568
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   order_id               96569 non-null  int64  
 1   customer_city          96569 non-null  object 
 2   customer_state         96569 non-null  object 
 3   seller_city            96569 non-null  object 
 4   seller_state           96569 non-null  object 
 5   order_item_id          96569 non-null  float64
 6   product_id             96569 non-null  float64
 7   seller_id              96569 non-null  float64
 8   price                  96569 non-null  float64
 9   freight_value          96569 non-null  float64
 10  payment_type           96569 non-null  float64
 11  payment_installments   96569 non-null  float64
 12  payment_value          96569 non-null  float64
 13  customer_unique_id     96569 non-null  float64
 14  product_category_name  96569 non-null  float64
 15  pr

In [65]:
#verificando se existe algum dado nulo
df_aux_orders_seller.isnull().sum()

order_id                 0
customer_city            0
customer_state           0
seller_city              0
seller_state             0
order_item_id            0
product_id               0
seller_id                0
price                    0
freight_value            0
payment_type             0
payment_installments     0
payment_value            0
customer_unique_id       0
product_category_name    0
product_weight_g         0
product_length_cm        0
product_height_cm        0
product_width_cm         0
costumer_lat             0
costumer_lng             0
seller_lat               0
seller_lng               0
days_to_delivery         0
early_delivery           0
day_shipping             0
month_shipping           0
year_shipping            0
day_order_approved       0
month_order_approved     0
year_order_approved      0
value                    0
dtype: int64

In [66]:
#criando filtro para remover colunas que nao atendem ao esperado
#nesse, caso deseja-se apenas as cidades com mais de mil pedidos
filtro  = df_aux_orders_seller['value'] > 1000

#cria-se df com as cidades que possuem mais de mil pedidos 
df_milorders_seller = df_aux_orders_seller[filtro]

In [67]:
df_milorders_seller.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56501 entries, 654 to 78432
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   order_id               56501 non-null  int64  
 1   customer_city          56501 non-null  object 
 2   customer_state         56501 non-null  object 
 3   seller_city            56501 non-null  object 
 4   seller_state           56501 non-null  object 
 5   order_item_id          56501 non-null  float64
 6   product_id             56501 non-null  float64
 7   seller_id              56501 non-null  float64
 8   price                  56501 non-null  float64
 9   freight_value          56501 non-null  float64
 10  payment_type           56501 non-null  float64
 11  payment_installments   56501 non-null  float64
 12  payment_value          56501 non-null  float64
 13  customer_unique_id     56501 non-null  float64
 14  product_category_name  56501 non-null  float64
 15  

In [68]:
df_milorders_seller.head(10)

,order_id,customer_city,customer_state,seller_city,seller_state,order_item_id,product_id,seller_id,price,freight_value,...,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value
654,72494,campos dos goytacazes,RJ,ibitinga,SP,1.0,9464.0,2340.0,102.00,14.07,...,-45.550141,1.0,18.0,21.0,3.0,2018.0,15.0,3.0,2018.0,5956
655,71772,campos dos goytacazes,RJ,ibitinga,SP,1.0,6698.0,579.0,79.99,8.75,...,-46.549510,1.0,17.0,6.0,2.0,2018.0,29.0,1.0,2018.0,5956
656,71772,campos dos goytacazes,RJ,ibitinga,SP,2.0,6698.0,579.0,79.99,8.75,...,-46.549510,1.0,17.0,6.0,2.0,2018.0,29.0,1.0,2018.0,5956
657,3374,belo horizonte,MG,ibitinga,SP,1.0,29463.0,1329.0,49.99,12.48,...,-48.192254,1.0,17.0,3.0,9.0,2017.0,28.0,8.0,2017.0,5956
658,92936,conselheiro lafaiete,MG,ibitinga,SP,1.0,22429.0,1049.0,89.00,35.24,...,-48.795940,6.0,29.0,19.0,4.0,2017.0,13.0,4.0,2017.0,5956
659,70310,conselheiro lafaiete,MG,ibitinga,SP,1.0,8337.0,2755.0,202.89,16.17,...,-49.625339,1.0,21.0,30.0,6.0,2017.0,26.0,6.0,2017.0,5956
660,52537,conselheiro lafaiete,MG,ibitinga,SP,1.0,18720.0,1308.0,134.90,29.34,...,-50.065021,0.0,25.0,7.0,3.0,2018.0,1.0,3.0,2018.0,5956
661,50876,conselheiro lafaiete,MG,ibitinga,SP,1.0,14172.0,560.0,89.90,18.51,...,-46.575615,6.0,21.0,22.0,3.0,2018.0,16.0,3.0,2018.0,5956
662,42966,conselheiro lafaiete,MG,ibitinga,SP,1.0,864.0,2796.0,259.00,42.79,...,-47.723450,3.0,32.0,3.0,9.0,2018.0,14.0,8.0,2018.0,5956
663,81265,conselheiro lafaiete,MG,ibitinga,SP,1.0,15469.0,1310.0,65.60,14.55,...,-49.365085,1.0,15.0,6.0,4.0,2018.0,2.0,4.0,2018.0,5956


In [69]:
import plotly.express as px

fig = px.choropleth_mapbox(df_milorders_seller, geojson=cidades, color='days_to_delivery', range_color=(0, 20),
                           locations=('seller_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3, hover_name="value")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

######Mais de vinte pedidos por cidade

In [70]:
filtro  = df_aux_orders_seller['value'] > 20
df_20orders_seller = df_aux_orders_seller[filtro]

In [72]:
import plotly.express as px

fig = px.choropleth_mapbox(df_20orders_seller, geojson=cidades, color='days_to_delivery', range_color=(0, 20),
                           locations=('seller_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3, hover_name="value")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

###### Mapa para numero de pedidos por cidade

In [73]:
import plotly.express as px

fig = px.choropleth_mapbox(df_aux_orders_seller, geojson=cidades, color='value', range_color=(0, 20),
                           locations=('seller_city'), featureidkey="properties.name", 
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3, hover_name="value")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

###Days to delivery pela media dos dias para uma cidade

In [74]:
mean_days_seller = df_aux_orders_seller.groupby(['seller_city']).mean()

In [75]:
#transformando em um pandas dataframe
mean_days_seller = pd.DataFrame(mean_days_seller)

In [76]:
mean_days_seller.head(5)

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value
seller_city,,,,,,,,,,,,,,,,,,,,,
04482255,83618.000,1.50,17997.500000,80.000000,223.48500,18.270000,0.000000,1.000000,483.510000,72369.000000,...,-51.161793,2.0,31.000,23.000,5.000000,2018.00,19.000000,5.000,2018.00,2.0
abadia de goias,35309.000,1.00,11216.000000,208.000000,199.00000,21.710000,0.000000,1.000000,220.710000,16496.000000,...,-51.468730,6.0,27.000,29.000,3.000000,2017.00,16.000000,3.000,2017.00,1.0
afonso claudio,67119.500,1.50,10454.166667,1191.166667,69.26500,8.801667,0.333333,1.666667,137.323333,46237.666667,...,-48.071320,2.0,19.500,12.000,5.666667,2018.00,11.833333,5.500,2018.00,6.0
aguas claras df,5530.000,1.00,23434.000000,237.000000,179.90000,21.650000,1.000000,1.000000,201.550000,84647.000000,...,-46.573341,2.0,22.000,19.000,4.000000,2018.00,14.000000,4.000,2018.00,1.0
alambari,58884.375,1.75,16631.375000,1641.750000,41.49875,14.003750,1.000000,1.000000,87.768750,32951.500000,...,-47.038788,1.5,20.125,16.125,5.500000,2017.75,15.375000,5.375,2017.75,8.0


In [79]:
mean_days_seller['seller_city'] = mean_days_seller.index

In [81]:
mean_days_seller.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value,seller_city
seller_city,,,,,,,,,,,,,,,,,,,,,
04482255,83618.000,1.50,17997.500000,80.000000,223.48500,18.270000,0.000000,1.000000,483.510000,72369.000000,...,2.0,31.000,23.000,5.000000,2018.00,19.000000,5.000,2018.00,2.0,04482255
abadia de goias,35309.000,1.00,11216.000000,208.000000,199.00000,21.710000,0.000000,1.000000,220.710000,16496.000000,...,6.0,27.000,29.000,3.000000,2017.00,16.000000,3.000,2017.00,1.0,abadia de goias
afonso claudio,67119.500,1.50,10454.166667,1191.166667,69.26500,8.801667,0.333333,1.666667,137.323333,46237.666667,...,2.0,19.500,12.000,5.666667,2018.00,11.833333,5.500,2018.00,6.0,afonso claudio
aguas claras df,5530.000,1.00,23434.000000,237.000000,179.90000,21.650000,1.000000,1.000000,201.550000,84647.000000,...,2.0,22.000,19.000,4.000000,2018.00,14.000000,4.000,2018.00,1.0,aguas claras df
alambari,58884.375,1.75,16631.375000,1641.750000,41.49875,14.003750,1.000000,1.000000,87.768750,32951.500000,...,1.5,20.125,16.125,5.500000,2017.75,15.375000,5.375,2017.75,8.0,alambari


In [83]:
import plotly.express as px

fig = px.choropleth_mapbox(mean_days_seller, geojson=cidades, color='days_to_delivery', range_color=(0, 5),
                           locations=('seller_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [84]:
#criando filtro para remover colunas que nao atendem ao esperado
#nesse, caso deseja-se apenas as cidades com mais de mil pedidos
filtro  = mean_days_seller['value'] > 20

#cria-se df com as cidades que possuem mais de mil pedidos 
df_20orders_mean_seller = mean_days_seller[filtro]

In [85]:
import plotly.express as px

fig = px.choropleth_mapbox(df_20orders_mean_seller, geojson=cidades, color='days_to_delivery', range_color=(0, 7),
                           locations=('seller_city'), featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [86]:
mean_days_seller.info()

<class 'pandas.core.frame.DataFrame'>
Index: 586 entries, 04482255 to xaxim
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   order_id               586 non-null    float64
 1   order_item_id          586 non-null    float64
 2   product_id             586 non-null    float64
 3   seller_id              586 non-null    float64
 4   price                  586 non-null    float64
 5   freight_value          586 non-null    float64
 6   payment_type           586 non-null    float64
 7   payment_installments   586 non-null    float64
 8   payment_value          586 non-null    float64
 9   customer_unique_id     586 non-null    float64
 10  product_category_name  586 non-null    float64
 11  product_weight_g       586 non-null    float64
 12  product_length_cm      586 non-null    float64
 13  product_height_cm      586 non-null    float64
 14  product_width_cm       586 non-null    float64
 15  co

##### Mapa para media de numero de pedidos por cidade

In [87]:
import plotly.express as px

fig = px.choropleth_mapbox(mean_days_seller, geojson=cidades, color='value', range_color=(0, 20),
                           locations=('seller_city'), featureidkey="properties.name", 
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

###Days to delivery pelo desvio padrão dos dias para uma cidade

In [88]:
desvio_seller = df_aux_orders_seller.groupby(['seller_city']).std()

In [89]:
desvio_seller = pd.DataFrame(desvio_seller)

In [90]:
desvio_seller.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value
seller_city,,,,,,,,,,,,,,,,,,,,,
04482255,0.000000,0.707107,8396.893027,0.000000,94.038131,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.0
abadia de goias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
afonso claudio,19485.725193,0.836660,7787.588418,1226.317808,54.927768,5.135541,0.516398,1.21106,67.093409,20380.224098,...,1.453814,2.000000,11.827933,8.000000,1.505545,0.00000,10.740888,1.224745,0.00000,0.0
aguas claras df,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
alambari,29329.049563,1.164965,9731.882301,932.051616,36.634391,5.104197,0.000000,0.00000,13.242701,6093.553362,...,2.231558,1.069045,10.148012,7.376362,2.725541,0.46291,6.345696,2.875388,0.46291,0.0


In [91]:
#verificando se existe algum dado nulo
desvio_seller.isnull().sum()

order_id                 63
order_item_id            63
product_id               63
seller_id                63
price                    63
freight_value            63
payment_type             63
payment_installments     63
payment_value            63
customer_unique_id       63
product_category_name    63
product_weight_g         63
product_length_cm        63
product_height_cm        63
product_width_cm         63
costumer_lat             63
costumer_lng             63
seller_lat               63
seller_lng               63
days_to_delivery         63
early_delivery           63
day_shipping             63
month_shipping           63
year_shipping            63
day_order_approved       63
month_order_approved     63
year_order_approved      63
value                    63
dtype: int64

In [92]:
desvio_seller.dropna(inplace=True)

In [93]:
desvio_seller.isnull().sum()

order_id                 0
order_item_id            0
product_id               0
seller_id                0
price                    0
freight_value            0
payment_type             0
payment_installments     0
payment_value            0
customer_unique_id       0
product_category_name    0
product_weight_g         0
product_length_cm        0
product_height_cm        0
product_width_cm         0
costumer_lat             0
costumer_lng             0
seller_lat               0
seller_lng               0
days_to_delivery         0
early_delivery           0
day_shipping             0
month_shipping           0
year_shipping            0
day_order_approved       0
month_order_approved     0
year_order_approved      0
value                    0
dtype: int64

In [94]:
desvio_seller.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value
seller_city,,,,,,,,,,,,,,,,,,,,,
04482255,0.000000,0.707107,8396.893027,0.000000,94.038131,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
afonso claudio,19485.725193,0.836660,7787.588418,1226.317808,54.927768,5.135541,0.516398,1.211060,67.093409,20380.224098,...,1.453814,2.000000,11.827933,8.000000,1.505545,0.000000,10.740888,1.224745,0.000000,0.0
alambari,29329.049563,1.164965,9731.882301,932.051616,36.634391,5.104197,0.000000,0.000000,13.242701,6093.553362,...,2.231558,1.069045,10.148012,7.376362,2.725541,0.462910,6.345696,2.875388,0.462910,0.0
alfenas,24964.670324,0.291542,9328.741753,815.504711,97.156711,11.319052,0.633077,2.548323,104.441045,27316.214907,...,2.372586,2.224608,9.868954,7.815476,3.061578,0.503166,8.478167,3.159455,0.503166,0.0
alvares machado,31104.478300,0.377964,11076.748597,843.532538,39.671894,1.880598,0.487950,1.112697,35.774422,35349.040547,...,2.106144,4.017817,5.740416,8.599557,3.184785,0.487950,6.848705,3.184785,0.487950,0.0


In [95]:
desvio_seller['seller_city'] = desvio_seller.index

In [96]:
import plotly.express as px

fig = px.choropleth_mapbox(desvio_seller, geojson=cidades, color='days_to_delivery', range_color=(0, 10),
                           locations=('seller_city'), featureidkey="properties.name", 
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

##State

In [97]:
db.head()

,order_id,customer_city,customer_state,seller_city,seller_state,order_item_id,product_id,seller_id,price,freight_value,...,seller_lat,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved
0,0,campos dos goytacazes,RJ,volta redonda,SP,1.0,8216.0,812.0,58.90,13.29,...,-22.497188,-44.127324,6.0,10.0,19.0,9.0,2017.0,13.0,9.0,2017.0
1,7310,jatai,GO,volta redonda,SP,1.0,28330.0,617.0,72.00,17.78,...,-16.690438,-49.327015,0.0,22.0,10.0,10.0,2017.0,4.0,10.0,2017.0
2,31905,belo horizonte,MG,volta redonda,SP,1.0,28451.0,1444.0,99.99,17.43,...,-23.486138,-46.367107,5.0,12.0,30.0,8.0,2017.0,18.0,8.0,2017.0
3,42787,sao jose dos pinhais,PR,volta redonda,SP,1.0,29612.0,1901.0,69.90,9.34,...,-23.490646,-46.476651,7.0,9.0,14.0,2.0,2018.0,10.0,2.0,2018.0
4,42787,sao jose dos pinhais,PR,volta redonda,SP,1.0,29612.0,1901.0,69.90,9.34,...,-23.490646,-46.476651,7.0,9.0,14.0,2.0,2018.0,10.0,2.0,2018.0


####Fazendo filtro no mapa



###Days to delivery pela media dos dias para uma cidade

In [98]:
mean_days_state_seller = df_aux_orders_seller.groupby(['seller_state']).mean()

In [99]:
#transformando em um pandas dataframe
mean_days_state_sellere = pd.DataFrame(mean_days_state_seller)

In [100]:
mean_days_state_seller.head(5)

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value
seller_state,,,,,,,,,,,,,,,,,,,,,
AC,81365.000000,1.000000,21292.000000,2352.000000,15.000000,15.100000,2.000000,1.000000,15.050000,19480.000000,...,-46.624522,2.000000,22.000000,29.000000,8.000000,2017.000000,22.000000,8.000000,2017.000000,2.000000
AM,73862.500000,1.000000,24605.500000,1546.000000,38.445000,12.160000,1.000000,3.000000,50.605000,86217.500000,...,-46.945001,2.000000,9.000000,12.000000,8.000000,2017.500000,22.000000,7.500000,2017.500000,2.000000
BA,45899.973333,1.150476,16402.489524,1432.763810,118.183848,19.958724,0.857143,2.706667,151.782762,44001.832381,...,-47.357423,2.607619,19.350476,16.045714,6.194286,2017.533333,15.580952,6.144762,2017.521905,226.470476
CE,47923.360465,1.069767,14775.000000,1592.848837,140.130930,22.298372,0.988372,3.023256,170.383721,48449.209302,...,-47.457538,2.918605,19.906977,15.581395,6.151163,2017.534884,14.883721,5.976744,2017.534884,35.302326
DF,48620.931122,1.184949,15546.655612,1442.330357,131.975918,19.494630,0.867347,3.169643,183.472181,46444.622449,...,-47.216873,2.746173,18.678571,16.301020,6.163265,2017.549745,15.301020,6.056122,2017.544643,704.201531


In [101]:
mean_days_state_seller['seller_state'] = mean_days_state_seller.index

In [102]:
mean_days_state_seller.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value,seller_state
seller_state,,,,,,,,,,,,,,,,,,,,,
AC,81365.000000,1.000000,21292.000000,2352.000000,15.000000,15.100000,2.000000,1.000000,15.050000,19480.000000,...,2.000000,22.000000,29.000000,8.000000,2017.000000,22.000000,8.000000,2017.000000,2.000000,AC
AM,73862.500000,1.000000,24605.500000,1546.000000,38.445000,12.160000,1.000000,3.000000,50.605000,86217.500000,...,2.000000,9.000000,12.000000,8.000000,2017.500000,22.000000,7.500000,2017.500000,2.000000,AM
BA,45899.973333,1.150476,16402.489524,1432.763810,118.183848,19.958724,0.857143,2.706667,151.782762,44001.832381,...,2.607619,19.350476,16.045714,6.194286,2017.533333,15.580952,6.144762,2017.521905,226.470476,BA
CE,47923.360465,1.069767,14775.000000,1592.848837,140.130930,22.298372,0.988372,3.023256,170.383721,48449.209302,...,2.918605,19.906977,15.581395,6.151163,2017.534884,14.883721,5.976744,2017.534884,35.302326,CE
DF,48620.931122,1.184949,15546.655612,1442.330357,131.975918,19.494630,0.867347,3.169643,183.472181,46444.622449,...,2.746173,18.678571,16.301020,6.163265,2017.549745,15.301020,6.056122,2017.544643,704.201531,DF


In [104]:
import plotly.express as px

fig = px.choropleth_mapbox(mean_days_state_seller, geojson=counties, color='days_to_delivery', range_color=(0, 5),
                           locations=('seller_state'), featureidkey="properties.SIGLA", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

###Days to delivery pelo desvio padrão dos dias para estado

In [111]:
desvio_seller_state = df_aux_orders_seller.groupby(['seller_state']).std()

In [112]:
desvio_seller_state = pd.DataFrame(desvio_seller_state)

In [113]:
desvio_seller_state.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value
seller_state,,,,,,,,,,,,,,,,,,,,,
AC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.414214,0.000000,14.071425,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AM,11478.464379,0.000000,5419.973478,1446.740474,15.492710,6.194255,0.000000,0.000000,21.686965,296.277741,...,0.373991,0.000000,0.000000,12.727922,1.414214,0.707107,7.071068,2.121320,0.707107,0.000000
BA,27145.951874,0.554497,8946.884870,830.579556,180.236674,15.111768,0.586563,2.509838,159.680592,25873.377483,...,2.820274,3.000372,8.539257,8.551026,3.233349,0.503171,8.852891,3.254181,0.503799,131.860040
CE,27657.153938,0.256249,7728.873982,890.965458,217.934556,20.935577,0.641582,2.794856,236.489338,27184.254254,...,2.369916,5.269703,8.801909,8.455371,2.703015,0.501707,8.967154,2.790643,0.501707,15.846373
DF,27791.542822,0.619173,9182.090308,835.291348,235.815588,15.098815,0.585264,2.913052,266.015543,26428.428655,...,2.163535,3.206493,8.852776,8.536524,3.259548,0.500396,8.655826,3.271755,0.500877,158.989442


In [114]:
#verificando se existe algum dado nulo
desvio_seller_state.isnull().sum()

order_id                 0
order_item_id            0
product_id               0
seller_id                0
price                    0
freight_value            0
payment_type             0
payment_installments     0
payment_value            0
customer_unique_id       0
product_category_name    0
product_weight_g         0
product_length_cm        0
product_height_cm        0
product_width_cm         0
costumer_lat             0
costumer_lng             0
seller_lat               0
seller_lng               0
days_to_delivery         0
early_delivery           0
day_shipping             0
month_shipping           0
year_shipping            0
day_order_approved       0
month_order_approved     0
year_order_approved      0
value                    0
dtype: int64

In [115]:
desvio_seller_state.dropna(inplace=True)

In [116]:
desvio_seller_state.isnull().sum()

order_id                 0
order_item_id            0
product_id               0
seller_id                0
price                    0
freight_value            0
payment_type             0
payment_installments     0
payment_value            0
customer_unique_id       0
product_category_name    0
product_weight_g         0
product_length_cm        0
product_height_cm        0
product_width_cm         0
costumer_lat             0
costumer_lng             0
seller_lat               0
seller_lng               0
days_to_delivery         0
early_delivery           0
day_shipping             0
month_shipping           0
year_shipping            0
day_order_approved       0
month_order_approved     0
year_order_approved      0
value                    0
dtype: int64

In [117]:
desvio_seller_state.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved,value
seller_state,,,,,,,,,,,,,,,,,,,,,
AC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.414214,0.000000,14.071425,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AM,11478.464379,0.000000,5419.973478,1446.740474,15.492710,6.194255,0.000000,0.000000,21.686965,296.277741,...,0.373991,0.000000,0.000000,12.727922,1.414214,0.707107,7.071068,2.121320,0.707107,0.000000
BA,27145.951874,0.554497,8946.884870,830.579556,180.236674,15.111768,0.586563,2.509838,159.680592,25873.377483,...,2.820274,3.000372,8.539257,8.551026,3.233349,0.503171,8.852891,3.254181,0.503799,131.860040
CE,27657.153938,0.256249,7728.873982,890.965458,217.934556,20.935577,0.641582,2.794856,236.489338,27184.254254,...,2.369916,5.269703,8.801909,8.455371,2.703015,0.501707,8.967154,2.790643,0.501707,15.846373
DF,27791.542822,0.619173,9182.090308,835.291348,235.815588,15.098815,0.585264,2.913052,266.015543,26428.428655,...,2.163535,3.206493,8.852776,8.536524,3.259548,0.500396,8.655826,3.271755,0.500877,158.989442


In [118]:
desvio_seller_state['seller_state'] = desvio_seller_state.index

In [120]:
import plotly.express as px

fig = px.choropleth_mapbox(desvio_seller_state, geojson=counties, color='days_to_delivery', range_color=(0, 5),
                           locations=('seller_state'), featureidkey="properties.SIGLA", 
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()